In [2]:
import numpy as np
import PIL
from PIL import Image
from sklearn import datasets
from sklearn.model_selection import train_test_split

# 1. Download the Dataset and Understand the Format

In [ ]:
#Downloaded in the project directory

# 2. Generate the Data Matrix and the Label vector

In [7]:
i = 0
s = 1
x = 1
d = [0] * 400
y = [0] * 400

while i < 400:
  image = PIL.Image.open("dataset/s" + str(s) + "/" + str(x) + ".pgm")
  image_sequence = image.getdata()
  image_array = np.array(image_sequence)
  d[i] = image_array
  y[i] = s
  i = i + 1
  x = x + 1
  if x == 11:
    s = s + 1
    x = 1

400
10304


# 3. Split the Dataset into Training and Test sets

In [48]:
training_data = [0] * 200
testing_data = [0] * 200
training_labels = [0] * 200
testing_labels = [0] * 200
curr_index = 0

In [49]:
for i in range(1, 400, 2):
  curr_index = int(i / 2);
  testing_data[curr_index] = d[i]
  testing_labels[curr_index] = y[i]
  training_data[curr_index] = d[i - 1]
  training_labels[curr_index] = y[i - 1]

testing_data = np.array(testing_data)
testing_labels = np.array(testing_labels)
training_data = np.array(training_data)
training_labels = np.array(training_labels)

# 4. Classification using PCA

# 5. Classification Using LDA 

In [6]:
def center_function(d):
    m = np.mean(d, axis=0)
    return np.subtract(d, m)

In [7]:
def lda(data, l, dom, n_class, features):
  # Calculate the mean of feature for every class
  # Mean matrix is 10304x1 dim
  cdata = []
  for i in range(0, len(data), l):
    cdata.append(data[i:i+l])
  mean_c = []
  for i in range(n_class):
    mean_c.append(np.mean(cdata[i], axis=0))
  if n_class != 2:
    tmean = np.mean(data, axis=0) 

  # Calculate between-class scatter matrix
  # B matrix is 10304 x 10304
  bmat = np.zeros((features, features))
  if n_class == 2:
    sub = np.subtract(mean_c[0], mean_c[1])
    bmat = np.dot(sub, sub.T)
  else:
    tm = tmean[:, np.newaxis]
    for i in range(n_class):
      mc = mean_c[i][:, np.newaxis]
      sub = np.subtract(mc, tm)
      bmat += np.multiply(l, np.dot(sub, sub.T))

  # Center class matrix 
  # Zi matrix is 5 x 10304
  zmat = []
  for d in cdata:
    zmat.append(center_function(d))
  
  # Class scatter matrix 
  # S matrix is 10304 x 10304
  s_tot = np.zeros((features, features))
  for zi in zmat:
    s_tot += np.dot(zi.T, zi)
 
  # Compute dominant eigenvector
  eval, evec = np.linalg.eigh(np.dot(np.linalg.inv(s_tot), bmat))

  # Return the 39 dominant eigenvectors and the mean
  idx = eval.argsort()[::-1]
  eva = eval[idx]
  eve = evec[:,idx]
  dom_arr = eve[:,:dom]
  
  return dom_arr.T

In [8]:
pj_mat = lda(d, 5, 39, 40, 10304)

In [9]:
pj_mat.shape

(39, 10304)

In [10]:
d.shape

(200, 10304)

In [11]:
new_train = np.dot(training_data, pj_mat.T)
new_train.shape

(200, 39)

In [12]:
new_test = np.dot(testing_data, pj_mat.T)
new_test.shape

(200, 39)

In [13]:
from sklearn.neighbors import KNeighborsClassifier

In [14]:
model_1_lda = KNeighborsClassifier(n_neighbors=1)
model_3_lda = KNeighborsClassifier(n_neighbors=3)
model_5_lda = KNeighborsClassifier(n_neighbors=5)
model_7_lda = KNeighborsClassifier(n_neighbors=7)

In [15]:
model_1_lda.fit(new_train, training_labels)
model_3_lda.fit(new_train, training_labels)
model_5_lda.fit(new_train, training_labels)
model_7_lda.fit(new_train, training_labels)

KNeighborsClassifier(n_neighbors=7)

In [16]:
pred_model_1_lda = model_1_lda.predict(new_test)
pred_model_3_lda = model_3_lda.predict(new_test)
pred_model_5_lda = model_5_lda.predict(new_test)
pred_model_7_lda = model_7_lda.predict(new_test)
pred_model_1_lda

array([ 1,  1,  1,  1, 32,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,
        4,  4,  4,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  7,  7,  7,  7,
        7,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9, 10, 10, 10, 10,  8, 11,
       11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14,
       14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17,
       18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 21, 21,
       21, 21, 21, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24,
       24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28,
       28, 28, 37, 28, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31,
       31, 31, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34,
        5, 15, 35, 25, 40, 36, 36, 36, 36, 17, 37, 37, 37, 37, 37, 38, 38,
       38, 23, 38, 39, 39, 39, 39, 39, 40,  5, 40, 40,  5])

In [17]:
testing_labels

array([ 1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,
        4,  4,  4,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  7,  7,  7,  7,
        7,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 11,
       11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14,
       14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17,
       18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 21, 21,
       21, 21, 21, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24,
       24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28,
       28, 28, 28, 28, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31,
       31, 31, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34,
       35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 38, 38,
       38, 38, 38, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40])

In [20]:
from sklearn.metrics import accuracy_score
acc_1_lda = accuracy_score(testing_labels, pred_model_1_lda)
acc_3_lda = accuracy_score(testing_labels, pred_model_3_lda)
acc_5_lda = accuracy_score(testing_labels, pred_model_5_lda)
acc_7_lda = accuracy_score(testing_labels, pred_model_7_lda)
print(acc_1_lda)
print(acc_3_lda)
print(acc_5_lda)
print(acc_7_lda)

0.945
0.885
0.83
0.785


# 6. Classifier Tuning 

# 7. Compare vs Non-Face Images

# 8. Bonus 